<a href="https://colab.research.google.com/github/DJCordhose/ml-workshop/blob/master/notebooks/process/flask-server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serving using Flask

Links
* making flask server directly run on Colab: https://stackoverflow.com/questions/54465816/how-to-use-flask-in-google-colaboratory-python-notebook

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
!ls -l

total 280
drwxr-xr-x 2 root root   4096 Jul 22 14:43 models
-rw-r--r-- 1 root root 277537 Jul 22 14:42 rf.model
drwxr-xr-x 1 root root   4096 Jul  3 16:14 sample_data


In [0]:
import pickle

# loading models only once on startup
model = pickle.load(open('models/rf.model', 'rb'))

# speed_std_scale = pickle.load(open('models/speed_std_scale.model', 'rb'))
# age_std_scale = pickle.load(open('models/age_std_scale.model', 'rb'))
# miles_std_scale = pickle.load(open('models/miles_std_scale.model', 'rb'))

def predict(speed, age, miles):
#     sample = [[speed_std_scale.transform([speed])[0],
#                age_std_scale.transform([age])[0],
#                miles_std_scale.transform([miles])[0]]]
    sample = [[speed, age, miles]]

    result = int(model.predict(sample)[0])
    prediction = model.predict_proba(sample)[0].tolist()
    
    return result, prediction

In [13]:
predict(100, 48, 10)

(1, [0.07023809523809524, 0.6569444444444444, 0.2728174603174603])

In [14]:
from flask import Flask, request, jsonify
import socket
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

app = Flask(__name__)

@app.route("/ping")
def ping():
    return "pong"

@app.route('/predict', methods=['GET', 'POST'])
def do_predict():
    speed = request.json['speed']
    age = request.json['age']
    miles = request.json['miles']

    predicted_category, probabilities = predict(speed, age, miles)

    response = {
        'category': predicted_category,
        'prediction': probabilities,
    }
    return jsonify(response)
  
import threading
threading.Thread(target=app.run, kwargs={'host': '0.0.0.0', 'port': 80}).start()  

172.28.0.2
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [15]:
import requests
r = requests.get("http://172.28.0.2/ping")
print(r.status_code)
print(r.encoding)
print(r.apparent_encoding)
print(r.text)

172.28.0.2 - - [22/Jul/2019 14:55:34] "GET /ping HTTP/1.1" 200 -


200
utf-8
ascii
pong


In [21]:
r = requests.post("http://172.28.0.2/predict", json={'speed': 100, 'age': 48, 'miles': 10})
r.status_code

172.28.0.2 - - [22/Jul/2019 14:57:03] "POST /predict HTTP/1.1" 200 -


200

In [22]:
response = r.json()
response

{'category': 1,
 'prediction': [0.07023809523809524, 0.6569444444444444, 0.2728174603174603]}

In [24]:
response['prediction']

[0.07023809523809524, 0.6569444444444444, 0.2728174603174603]